In [ ]:
import pandas as pd
import json
import os
import numpy as np
import requests
import warnings

In [ ]:
os.getcwd()

In [ ]:
df = pd.read_csv('Submissions.csv')

In [ ]:
df_list = list(df['CIK'].unique())

In [ ]:
df_list = num_to_cik(df_list)

In [ ]:
df_list

In [ ]:
def split_list_equally(input_list, num_splits):
    avg = len(input_list) / float(num_splits)
    output_list = []
    last = 0.0

    while last < len(input_list):
        output_list.append(input_list[int(last):int(last + avg)])
        last += avg

    return output_list

# Example usage
my_list = df_list
num_splits = 50
result = split_list_equally(my_list, num_splits)


In [ ]:
len(result[0])

In [ ]:
import random

# Your original list
original_list = df_list

# Shuffle the original list randomly
random.shuffle(original_list)

# Calculate the number of elements to select (20% of the total)
num_elements_to_select = int(len(original_list) * 0.02)

# Select the first 20% of the shuffled list to create a new list
new_list = original_list[:num_elements_to_select]

In [ ]:
len(df_list)

In [ ]:
df_new_list = num_to_cik(df_list)
merged_df = Convertor_Multiple(df_new_list)
na_maxed = na_max(merged_df,threshold)
df_num = frame_num_drop(na_maxed)
divided = df_asset_target(df_num)
merged = process_in_chunks(divided,lvl1,lvl2,shift,chunk)
df_targeted=df_target_dropna(merged,lvl1,lvl2,target_shift)
df_shuffled = df_targeted.sample(frac=1, random_state=42)

In [ ]:
def num_to_cik(num_list):
    newlist = []
    for i in num_list:
        if len(str(i))==4:
            newlist.append('companyfacts/CIK000000' + str(i)+'.json')
        if len(str(i))==5:
            newlist.append('companyfacts/CIK00000' + str(i)+'.json')
        if len(str(i))==6:
            newlist.append('companyfacts/CIK0000' + str(i)+'.json')
        if len(str(i))==7:
            newlist.append('companyfacts/CIK000' + str(i)+'.json')
    return newlist

In [ ]:
df_new_list = num_to_cik(result[0])

In [ ]:
def Convertor_Multiple(file_paths):
    all_dfs = []
    
    for idx, path in enumerate(file_paths, start=1):
        with open(path, 'r') as json_file:
            data = json.load(json_file)

        columns = set()
        values = {}
        
        try:
            cik = data['cik']
            entity_name = data['entityName']
            us_gaap_facts = data['facts']['us-gaap']
        except KeyError as e:
            print(f"Key {e} missing in file {path}. Skipping this file.")
            continue

        for key, value in us_gaap_facts.items():
            for unit_key in value['units'].keys():
                for entry in value['units'][unit_key]:
                    try:
                        if 'frame' in entry:
                            frame = entry['frame']
                            if frame.endswith('I'):
                                frame = frame[:-1]
                            row_key = (frame, cik)
                            columns.add(key)
                            values.setdefault(row_key, {}).update({
                                'CIK': cik,
                                'EntityName': entity_name,
                                'Form': entry['form'],
                                'FP': entry['fp'],
                                'End': entry['end'],
                                'Filed': entry['filed'],
                                'Frame': frame,
                                key: entry['val']
                            })
                    except KeyError as e:
                        print(f"Key {e} missing in entry for {key}. Skipping this entry.")

        df = pd.DataFrame.from_dict(values, orient='index')
        all_dfs.append(df)

    merged_df = pd.concat(all_dfs, ignore_index=True)
    return merged_df

In [ ]:
df = Convertor_Multiple(result[0])

In [ ]:
for i in range(0, 50):
    df = Convertor_Multiple(result[i])
    df.to_csv(f'data/df_{i}.csv', mode='w', index=False)

In [ ]:
merged_df = Convertor_Multiple(df_new_list)

In [ ]:
merged_df.to_parquet('merged_df_0.parquet')

In [ ]:
def df_info(df):
    # Calculate the percentage of missing values
    missing_percentage = ((df.isnull().sum().sum()) / (df.shape[0] * df.shape[1])).round(2)
    
    # Get the shape of the DataFrame
    shape = df.shape
    
    return missing_percentage, shape

In [ ]:
df_info(merged_df)

In [ ]:
len(merged_df['CIK'].unique())

In [ ]:
def na_max(df,threshold):

    missing_values = df.isnull().sum()

    # Calculate the percentage of missing values in each column
    missing_percentage = missing_values / len(df)

    # Get the columns where the missing percentage is greater than the threshold
    columns_to_drop = missing_percentage[missing_percentage > threshold].index

    # Drop the columns from the DataFrame
    df = df.drop(columns=columns_to_drop)
    return df

In [ ]:
na_maxed = na_max(merged_df,0.4)

In [ ]:
list(na_maxed.columns)

In [ ]:
na_maxed

In [ ]:
df_num

In [ ]:
df_num=df_num[['CIK', 'Frame', 'Assets', 'EarningsPerShareBasic','ProfitLoss']]

In [ ]:
df_num_1 = df_num[['CIK','Frame']]
df_num_2 = df_num[df_num.columns[2:]]
df_num_2 = df_num_2.div(df_num['Assets'], axis=0)
df_num_2 = df_num_2.drop('Assets',axis=1)
df_num = pd.concat([df_num_1,df_num_2],axis=1)

In [ ]:
df_num_1 = df_num[['CIK','Frame']]
df_num_2 = df_num[df_num.columns[2:]]

In [ ]:
df_num_2.isnull().any()

In [ ]:
print(len(df_num_2[df_num_2['Assets'].isna()]))
print(len(df_num_2[df_num_2['EarningsPerShareBasic'].isna()]))
print(len(df_num_2[df_num_2['ProfitLoss'].isna()]))

In [ ]:
len(df_num_2)

In [ ]:
daf = df_num[df_num['EarningsPerShareBasic'].isna()]

In [ ]:
daf[daf['CIK']==0001976927]

In [ ]:
daf.head(20)

In [ ]:
merged_df=merged_df[['CIK', 'Frame', 'Assets', 'EarningsPerShareBasic','ProfitLoss']]

In [ ]:
df = merged_df.copy()

In [ ]:
df = df.sort_values(by='Frame', ascending=True)
df = pd.pivot_table(df, index=['CIK', 'Frame'])
df = df.reset_index()

In [ ]:
df.to_csv('test.csv',mode='w', index=False)

In [ ]:
na_maxed

In [ ]:
print(len(na_maxed[na_maxed['Assets'].isna()]))
print(len(na_maxed[na_maxed['EarningsPerShareBasic'].isna()]))
print(len(na_maxed[na_maxed['ProfitLoss'].isna()]))

In [ ]:
df_num_2

In [47]:
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm  # Import tqdm

# Path to the directory with CSV files
csv_dir = 'data 2'

# List to hold dataframes
dataframes = []

# Get a list of CSV files
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Iterate over each CSV file in the directory with a progress bar
for csv_file in tqdm(csv_files, desc='Loading CSV files'):
    # Read the CSV file
    df = pd.read_csv(os.path.join(csv_dir, csv_file))
    # Append the dataframe to the list
    dataframes.append(df)

# Concatenate all dataframes across rows, creating new columns as needed
merged_df = pd.concat(dataframes, axis=0, ignore_index=True, sort=False)

# Save the merged dataframe as a Parquet file
parquet_file = 'merged_data.parquet'
pq.write_table(pa.Table.from_pandas(merged_df), parquet_file)

print(f'Merged Parquet file saved as {parquet_file}')


Merged Parquet file saved as merged_data.parquet


In [4]:
len(merged_df['CIK'].unique())

7535

In [5]:
merged_df.to_parquet('merged_df_1.parquet')

In [1]:
import pandas as pd

In [ ]:
df = pd.read_parquet('merged_df_1.parquet')

In [3]:
import dask.dataframe as dd

# Specify the path to your parquet file
parquet_file = 'merged_df_1.parquet'

# Read the parquet file with Dask
ddf = dd.read_parquet(parquet_file)

In [4]:
ddf

,CIK,EntityName,Form,FP,End,Filed,Frame,AccountsAndOtherReceivablesNetCurrent,AccountsPayableAndAccruedLiabilitiesCurrent,AccountsPayableTradeCurrent,AccrualForEnvironmentalLossContingenciesGross,AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax,AllowanceForDoubtfulAccountsReceivableCurrent,AssetRetirementObligation,Assets,AssetsCurrent,CashAndCashEquivalentsAtCarryingValue,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,ContractualObligation,ContractualObligationDueAfterFifthYear,ContractualObligationDueInFifthYear,ContractualObligationDueInFourthYear,ContractualObligationDueInNextTwelveMonths,ContractualObligationDueInSecondYear,ContractualObligationDueInThirdYear,ContractWithCustomerAssetNet,ContractWithCustomerAssetNetCurrent,ContractWithCustomerLiability,CustomerAdvancesCurrent,DebtAndCapitalLeaseObligations,DebtCurrent,DeferredIncomeTaxAssetsNet,DeferredIncomeTaxLiabilities,DeferredTaxAssetsDerivativeInstruments,DeferredTaxAssetsLiabilitiesNet,DeferredTaxAssetsNet,DeferredTaxAssetsOperatingLossCarryforwards,DeferredTaxAssetsTaxDeferredExpenseReservesAndAccruals,DeferredTaxLiabilitiesDerivatives,DeferredTaxLiabilitiesOther,DeferredTaxLiabilitiesPropertyPlantAndEquipment,DerivativeAssetFairValueGrossLiability,DerivativeAssets,DerivativeFairValueOfDerivativeAsset,DerivativeFairValueOfDerivativeLiability,DerivativeLiabilities,DerivativeLiabilityCollateralRightToReclaimCashOffset,DerivativeLiabilityFairValueGrossAsset,DueFromRelatedPartiesCurrent,DueToRelatedPartiesCurrent,EmployeeServiceShareBasedCompensationNonvestedAwardsTotalCompensationCostNotYetRecognized,EquityMethodInvestmentDifferenceBetweenCarryingAmountAndUnderlyingEquity,EquityMethodInvestmentSummarizedFinancialInformationCurrentAssets,EquityMethodInvestmentSummarizedFinancialInformationCurrentLiabilities,EquityMethodInvestmentSummarizedFinancialInformationNoncurrentAssets,EquityMethodInvestmentSummarizedFinancialInformationNoncurrentLiabilities,EquityMethodInvestments,FiniteLivedIntangibleAssetsAccumulatedAmortization,FiniteLivedIntangibleAssetsAmortizationExpenseNextTwelveMonths,FiniteLivedIntangibleAssetsAmortizationExpenseYearFive,FiniteLivedIntangibleAssetsAmortizationExpenseYearFour,FiniteLivedIntangibleAssetsAmortizationExpenseYearThree,FiniteLivedIntangibleAssetsAmortizationExpenseYearTwo,FiniteLivedIntangibleAssetsGross,FiniteLivedIntangibleAssetsNet,Goodwill,InventoryNet,InventoryNoncurrent,LettersOfCreditOutstandingAmount,LiabilitiesAndStockholdersEquity,LiabilitiesCurrent,LiabilitiesNoncurrent,LiabilitiesOtherThanLongtermDebtNoncurrent,LongTermDebtAndCapitalLeaseObligations,LongTermDebtMaturitiesRepaymentsOfPrincipalAfterYearFive,LongTermDebtMaturitiesRepaymentsOfPrincipalInNextTwelveMonths,LongTermDebtMaturitiesRepaymentsOfPrincipalInYearFive,LongTermDebtMaturitiesRepaymentsOfPrincipalInYearFour,LongTermDebtMaturitiesRepaymentsOfPrincipalInYearThree,LongTermDebtMaturitiesRepaymentsOfPrincipalInYearTwo,NetIncomeLoss,NoncurrentAssets,OperatingIncomeLoss,OperatingLeasesFutureMinimumPaymentsDue,OperatingLeasesFutureMinimumPaymentsDueCurrent,OperatingLeasesFutureMinimumPaymentsDueInFiveYears,OperatingLeasesFutureMinimumPaymentsDueInFourYears,OperatingLeasesFutureMinimumPaymentsDueInThreeYears,OperatingLeasesFutureMinimumPaymentsDueInTwoYears,OperatingLeasesFutureMinimumPaymentsDueThereafter,OtherAssetsCurrent,OtherAssetsMiscellaneousNoncurrent,OtherAssetsNoncurrent,OtherCommitment,OtherCommitmentDueAfterFifthYear,OtherCommitmentDueInFifthYear,OtherCommitmentDueInFourthYear,OtherCommitmentDueInNextTwelveMonths,OtherCommitmentDueInSecondYear,OtherCommitmentDueInThirdYear,OtherLiabilitiesCurrent,OtherLongTermDebtNoncurrent,PartnersCapitalAdjustedBalance1,PartnersCapitalAttributableToNoncontrollingInterest,PartnersCapitalIncludingPortionAttributableToNoncontrollingInterest,ProfitLoss,PropertyPlantAndEquipmentGross,PropertyP